# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb langchain langchain_community langchain_openai

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [4]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [5]:
llm = OpenAI(temperature=0)

In [6]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [7]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [8]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [9]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [11]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [12]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [13]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\ma0me\AppData\Local\Temp\ipykernel_31940\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [15]:
agent.invoke("What is the theme of Sonnet 18?")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to find the theme of Sonnet 18.
Action: Sonnets QA System
Action Input: "What is the theme of Sonnet 18?"
Observation:  The theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and how the subject's beauty will never fade, unlike the fleeting nature of summer.
Thought: I now know the final answer.
Final Answer: The theme of Sonnet 18 is the comparison of the subject's beauty to a summer's day.

> Finished chain.


{'input': 'What is the theme of Sonnet 18?',
 'output': "The theme of Sonnet 18 is the comparison of the subject's beauty to a summer's day."}

In [16]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should consider the differences between ruff and flake8 before making a decision.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.
Thought: Now that I know the differences, I can make a more informed decision.
Final Answer: Ruff has more rules and can automatically fix its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'Ruff has more rules and can automatically fix its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [17]:
tools = [
    Tool(
        name="State of Union QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnets address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [18]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [19]:
agent.invoke("What is the theme of Sonnet 18?")



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What is the theme of Sonnet 18?"
Observation:  The theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and how the subject's beauty will never fade, unlike the fleeting nature of summer.


> Finished chain.


{'input': 'What is the theme of Sonnet 18?',
 'output': " The theme of Sonnet 18 is the speaker's comparison of the subject to a summer's day and how the subject's beauty will never fade, unlike the fleeting nature of summer."}

In [20]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the pros and cons of each tool before making a decision.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.

3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. This can save you time and effort when fixing issues in your code.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, which mea

{'input': 'Why use ruff over flake8?',
 'output': "\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.\n\n3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. This can save you time and effort when fixing issues in your code.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, which means you don't have to install and configure them separately.\n\n5. More customizable configuration: Ruff allows for more customizable configuration

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [21]:
tools = [
    Tool(
        name="State of Union QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about the most recent sonnets address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [22]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [23]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Is that tool mentioned in the sonnets?"
)



> Entering new AgentExecutor chain...
 I should check if ruff uses Jupyter Notebooks and if that tool is mentioned in the sonnets.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: I should check if the tool mentioned in the sonnets is the same as the one used by ruff.
Action: State of Union QA System
Action Input: Is that tool mentioned in the sonnets?
Observation:  No, the sonnets do not mention any specific tool. They primarily focus on themes of love, beauty, and the passage of time.
Thought: I now know that the tool used by ruff is not mentioned in the sonnets.
Final Answer: No, the tool used by ruff is not mentioned in the sonnets.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Is that tool mentioned in the sonnets?',
 'output': 'No, the tool used by ruff is not mentioned in the sonnets.'}